In [6]:
import pandas as pd

df = pd.read_csv("../data/processed/cleaned_activities.csv")

df.head()


,account_id,SourceSystem,activity_date,who_id,opportunity_id,opportunity_stage,is_lead,types,Country,solution,activity_order,action_weight
0,0010L00001hVmFhQAK,SFDC_US,2022-07-25,0030L00001vlbHLQAY,NaN,no_opp,1,Follow Up,US,MRS,1.0,1.0
1,0010L00001hVmFhQAK,SFDC_US,2023-02-08,0034X00002xZlQtQAK,NaN,no_opp,1,Call,US,MRS,2.0,1.0
2,0010L00001hVmFhQAK,SFDC_US,2023-02-14,0030L00001vlbHLQAY,NaN,no_opp,1,Call,US,MRS,3.0,1.0
3,0010L00001hVmFhQAK,SFDC_US,2023-02-20,0030L00001vlbHLQAY,NaN,no_opp,2,Call,US,MRS,4.0,1.0
4,0010L00001hVmFhQAK,SFDC_US,2023-03-16,0034X00003GOUrFQAX,NaN,no_opp,1,Call,US,MRS,5.0,1.0


In [7]:
# 1) بناء جميع المسارات لكل الحسابات
paths_df = (
    df.sort_values(["account_id", "activity_order"])
      .groupby("account_id")
      .agg(
          Country=("Country", "first"),
          solution=("solution", "first"),
          path=("types", lambda x: " → ".join(x))
      )
      .reset_index()
)

paths_df.head()


,account_id,Country,solution,path
0,0010L00001hVmFhQAK,US,MRS,Follow Up → Call → Call → Call → Call → Call →...
1,0010L00001hVxd6QAC,US,MRS,Meeting → Meeting → Review → Email → Email → E...
2,0010L00001hVyJQQA0,US,MRS,Call → Call → Call → Call → Email → Call → Call
3,0010L00001hW1cAQAS,US,MRS,Call
4,0010L00001ijzeFQAQ,US,MRS,Email → Email → Email → Call → Call → Email → ...


In [9]:
top5_paths = (
    paths_df
    .groupby(["Country", "solution", "path"])
    .size()
    .reset_index(name="count")
    .sort_values(["Country", "solution", "count"], ascending=[True, True, False])
    .groupby(["Country", "solution"])
    .head(5)
)

top5_paths.head(20)


,Country,solution,path,count
0,AT,MRS,Email,22
2,AT,MRS,Email → Email,19
4,AT,MRS,Email → Email → Email → Email,6
3,AT,MRS,Email → Email → Email,5
7,AT,MRS,Email → Email → Email → Email → Email → Email ...,5
18,BE,Digital,Email → Email → Email → Email → Email → Email ...,1
27,BE,MRS,Email,18
34,BE,MRS,Email → Email,14
40,BE,MRS,Email → Email → Email,4
44,BE,MRS,Email → Email → Email → Email,3


In [10]:
top5_paths.to_csv(
    "../data/processed/top5_paths_by_country_solution.csv",
    index=False
)
